In [39]:
from pathlib import Path
import datetime as dt
import yfinance as yf
import pandas as pd
import os

# make sure we’re in repo root
os.chdir("/Users/ravish/Documents/Stock-Project")

DATA_RAW = Path("data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

TICKERS = ["AAPL", "MSFT", "AMZN", "GOOG", "TSLA"]

# define date range (20 years)
END = dt.date.today()
START = END - dt.timedelta(days=365*20 + 7)

print("Saving raw files to:", DATA_RAW.resolve())

Saving raw files to: /Users/ravish/Documents/Stock-Project/data/raw


In [40]:
from pathlib import Path

p = Path("data/raw/AAPL.csv")
print("Exists:", p.exists())
print("Size bytes:", p.stat().st_size if p.exists() else "-")
if p.exists() and p.stat().st_size > 0:
    print("First 5 lines:\n", "\n".join(p.read_text().splitlines()[:5]))

Exists: True
Size bytes: 558043
First 5 lines:
 Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2005-09-23,1.5970702171325684,1.899999976158142,1.910714030265808,1.8514289855957031,1.8607139587402344,558457200
2005-09-26,1.6162832975387573,1.9228570461273193,1.948570966720581,1.9042860269546509,1.9296430349349976,546562800


In [41]:
def fetch_and_save(ticker, start, end):
    df = yf.download(ticker, start=start, end=end, auto_adjust=False, progress=False)
    df.index.name = "Date"
    out = DATA_RAW / f"{ticker}.csv"   # simple filenames!
    df.to_csv(out)
    print(f"{ticker}: {len(df)} rows saved → {out}")
    return out

for t in TICKERS:
    fetch_and_save(t, START, END)

print("✅ All done")

AAPL: 5032 rows saved → data/raw/AAPL.csv
MSFT: 5032 rows saved → data/raw/MSFT.csv
AMZN: 5032 rows saved → data/raw/AMZN.csv
GOOG: 5032 rows saved → data/raw/GOOG.csv
TSLA: 3834 rows saved → data/raw/TSLA.csv
✅ All done


In [51]:
def load_raw(ticker: str):
    """
    Loads data/raw/{TICKER}.csv where the file has:
      header row: Price, Adj Close, Close, High, Low, Open, Volume
      row 0: Ticker, AAPL, AAPL, ...
      row 1: Date, , , , , ,
      row 2+: real data (first column is the date)
    """
    import pandas as pd
    from pathlib import Path

    p = DATA_RAW / f"{ticker}.csv"
    df = pd.read_csv(p)                 # read as-is

    # standardize column names
    df.columns = df.columns.str.strip()

    # first column currently called "Price" but contains 'Ticker', 'Date', then dates
    first = df.columns[0]

    # drop the two metadata rows
    df = df[~df[first].isin(["Ticker", "Date"])].copy()

    # rename first col to Date and parse to datetime
    df.rename(columns={first: "Date"}, inplace=True)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).set_index("Date").sort_index()

    # make numeric columns numeric
    for c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


def clean_stock(df: pd.DataFrame) -> pd.DataFrame:
    keep = ["Open","High","Low","Close","Adj Close","Volume"]
    df = df[[c for c in keep if c in df.columns]].copy()
    if "Adj Close" not in df.columns:
        raise KeyError(f"'Adj Close' not found. Columns are: {list(df.columns)}")
    df = df[~df.index.duplicated(keep="first")].ffill().dropna()
    return df

def add_basic_features(df: pd.DataFrame) -> pd.DataFrame:
    df["Return"] = df["Adj Close"].pct_change()
    df["CumReturn"] = (1 + df["Return"]).cumprod()
    return df

def process_and_save(ticker: str):
    raw = load_raw(ticker)
    before = raw.shape
    df = add_basic_features(clean_stock(raw))
    out = DATA_PROC / f"{ticker}_processed.csv"
    df.to_csv(out)
    print(f"✅ {ticker}: {before} → {df.shape}  saved → {out.name}")
    return ticker, df.shape, out.name


def add_basic_features(df: pd.DataFrame) -> pd.DataFrame:
    df["Return"] = df["Adj Close"].pct_change()
    df["CumReturn"] = (1 + df["Return"]).cumprod()
    return df


In [52]:
summary = []
for t in TICKERS:
    try:
        summary.append(process_and_save(t))
    except Exception as e:
        try:
            cols = list(load_raw(t).columns)
        except Exception:
            cols = "could not load"
        print(f"❌ {t} failed: {e}\n   Columns detected: {cols}")

import pandas as pd
pd.DataFrame(summary, columns=["Ticker","Shape","File"])

✅ AAPL: (5032, 6) → (5032, 8)  saved → AAPL_processed.csv
✅ MSFT: (5032, 6) → (5032, 8)  saved → MSFT_processed.csv
✅ AMZN: (5032, 6) → (5032, 8)  saved → AMZN_processed.csv
✅ GOOG: (5032, 6) → (5032, 8)  saved → GOOG_processed.csv
✅ TSLA: (3834, 6) → (3834, 8)  saved → TSLA_processed.csv


,Ticker,Shape,File
0,AAPL,"(5032, 8)",AAPL_processed.csv
1,MSFT,"(5032, 8)",MSFT_processed.csv
2,AMZN,"(5032, 8)",AMZN_processed.csv
3,GOOG,"(5032, 8)",GOOG_processed.csv
4,TSLA,"(3834, 8)",TSLA_processed.csv
